# KISSFLOW WEEKLY DISPLINARY ACTIONS REPORT

In [7]:
#Import Modules
import pandas as pd
from functools import reduce
from datetime import datetime
from datetime import timedelta 
import numpy as np
import os
os.chdir(r'C:\Users\user\Desktop\Analysis\KissFlow\2020_09_09')


In [8]:
#Import all items report data
AllItemData = pd.read_csv('Rwanda_Field_Team_disciplinary_actions.csv',
                          encoding='latin1',
                          usecols=['Case ID','Main case','First Name','Last Name','Region','District','Site','CreatedBy',
                                   'CreatedAt','Business_Step'])


In [9]:
#Import tasks report data
AllTasksData = pd.read_excel('AllTasks.xlsx') 

In [10]:
#Import cases under clarification
Clarify = pd.read_excel('Cases under clarification.xlsx')

### Cleaning task data

In [11]:
#Split Columns
SplitSubject = AllTasksData['Subject'].str.split("|", n = 5, expand = True)
SplitTaskName = AllTasksData['Task Name'].str.split("-", n = 2, expand = True)

#Change the date column of when a task has started to date type
AllTasksData['Started'] = AllTasksData['Started'].astype('datetime64[ns]')


#Assign split strings to data frame to create new columns
AllTasksData['Case ID']=SplitSubject[0]
AllTasksData['Case Type']=SplitSubject[1]
AllTasksData['Names']=SplitSubject[2]
AllTasksData['Payroll ID']=SplitSubject[3]
AllTasksData['Site']=SplitSubject[4]
AllTasksData['District']=SplitSubject[5]
AllTasksData['Department']=SplitTaskName[0]
AllTasksData['Task']=SplitTaskName[1]

#Correct departments naming to remove incorrect ones
AllTasksData['Department']=AllTasksData['Department'].replace(['Alice/Louise ','Confirmation of Legal Actions','FDV Manager assignement'],['Louise','Legal','FDV'])
AllTasksData['Task'] = AllTasksData['Task'].fillna('Confirmation of Legal Actions')
AllTasksData['Task'] = AllTasksData['Task'].str.strip()
AllTasksData['Department']= AllTasksData['Department'].str.strip()

#Cleaning cases under clarification
SplitItemSubject = Clarify['Item Subject'].str.split("|", n = 5, expand = True)
Clarify['Case ID']=SplitItemSubject[0]
Clarify['Case Type']=SplitItemSubject[1]
Clarify['Names']=SplitItemSubject[2]
Clarify['Payroll ID']=SplitItemSubject[3]
Clarify['Site']=SplitItemSubject[4]
Clarify['District']=SplitItemSubject[5]

### Report on total, completed and pending tasks

In [13]:
#Completed and pending taks per department
TotalTasks = AllTasksData.groupby('Department')['Case ID'].count().reset_index().rename(columns={'Case ID': 'Total Tasks Assigned'})
PendingTasks = AllTasksData[AllTasksData['completed?']=='No']
PendingTasks = PendingTasks .groupby('Department')['Case ID'].count().reset_index().rename(columns={'Case ID': 'Pending Tasks'})
CompletedTasks = AllTasksData[AllTasksData['completed?']=='Yes']
CompletedTasks = CompletedTasks.groupby('Department')['Case ID'].count().reset_index().rename(columns={'Case ID': 'Total completed Tasks'})

#Merge total, completed and pending tasks and prepare their percentages
AllDataReport = TotalTasks.merge(CompletedTasks,on='Department',how='outer').merge(PendingTasks,on='Department',how='outer')
AllDataReport['Pending Tasks'] = AllDataReport['Pending Tasks'].fillna(0)
AllDataReport['% Completed tasks']=AllDataReport['Total completed Tasks']*100/AllDataReport['Total Tasks Assigned']

#Tasks Under Clarification
ClarifyTasks = Clarify.groupby('Department')['Case ID'].count().reset_index()

AllTasksWithClarifications = pd.merge(AllDataReport,ClarifyTasks,on='Department',how='left').rename(columns={'Case ID':'Clarifications Requested'}).fillna(0) 
AllTasksWithClarifications['Total Pending Tasks'] = AllTasksWithClarifications['Pending Tasks'] +  AllTasksWithClarifications['Clarifications Requested'] 
AllTasksWithClarifications['% Pending tasks']=AllTasksWithClarifications['Total Pending Tasks']*100/AllTasksWithClarifications['Total Tasks Assigned']
AllTasksWithClarifications[['Department', 'Total Tasks Assigned', 'Total completed Tasks', 'Pending Tasks','Clarifications Requested','Total Pending Tasks','% Completed tasks','% Pending tasks']]

AllTasksWithClarifications.to_csv('Report on total, completed and pending tasks.csv')

AllTasksWithClarifications

,Department,Total Tasks Assigned,Total completed Tasks,Pending Tasks,% Completed tasks,Clarifications Requested,Total Pending Tasks,% Pending tasks
0,Christine,8,7,1,87.500000,0.0,1.0,12.500000
1,FDV,32,30,2,93.750000,0.0,2.0,6.250000
2,HR,152,141,11,92.763158,1.0,12.0,7.894737
3,Legal,10,9,1,90.000000,0.0,1.0,10.000000
4,Louise,68,65,3,95.588235,1.0,4.0,5.882353
5,RL,73,48,25,65.753425,1.0,26.0,35.616438


### Report on Tasks and SLAs

In [14]:
#Build SLA dataframe
SLAs = pd.DataFrame({'Task':['Confirmation of employee benefits',
                    'Update Roster',
                    'Confirmation of Legal Actions',
                    'Manager assignment',
                    'Approval of the decision taken',
                    'Confirm receipt of signed letter from staff',
                    'Initial Follow up on the case',
                    'To process the case',
                    'confirm a new employee',
                    'Write off clients credit',
                    'Decision implementation/submit documents signed by staff',
                    'Inform clients about new employee',
                    'Upload explanation letter from absent employee',
                    'hiring a new employee',
                    'Provision of Investigation results','Provide Clarification'],'SLA':[2,2,2,2,2,2,2,7,17,2,2,2,2,10,3,2]})

#Prepare pending tasks only data frame
RawAllPendingTasks = AllTasksData[AllTasksData['completed?']=='No']

#Merge all pending tasks with SLA
TasksWithSLA = pd.merge(RawAllPendingTasks,SLAs, on='Task', how='left')
ClarifyDf=Clarify[['Department','Deadline', 'Task', 'Case ID', 'Case Type', 'Names', 'Payroll ID','Site', 'District']]

#Add a deadline column
TasksWithSLA['Deadline'] = TasksWithSLA['Started']+pd.to_timedelta(TasksWithSLA['SLA'], unit='d')
TasksWithSLA = pd.concat([TasksWithSLA,ClarifyDf])

TasksWithSLA["Day"] = TasksWithSLA['Deadline'].dt.day_name()

#Add Days for tasks with deadline in the weekend
ConditionsForAddition = [(TasksWithSLA["Day"]=='Saturday'),((TasksWithSLA["Day"]=='Sunday'))]
ValuesForAddition = [2,2]
TasksWithSLA["Additional Days"]=np.select(ConditionsForAddition,ValuesForAddition)

#Final deadline
TasksWithSLA['Final Deadline'] = TasksWithSLA['Deadline']+pd.to_timedelta(TasksWithSLA['Additional Days'], unit='d')

In [15]:
Today = input('Add Today date:')

Add Today date:2020-09-09


In [16]:
SevereSLABreach = input('Add severe SLA breach date:')

Add severe SLA breach date:2020-08-31


In [19]:
#Prepare conditions for SLA status
conditions = [(TasksWithSLA['Final Deadline']>=Today),
              (TasksWithSLA['Final Deadline']<Today)&(TasksWithSLA['Final Deadline']>=SevereSLABreach),
              (TasksWithSLA['Final Deadline']<SevereSLABreach)]

#Values if a given condition is met
values = ['Within SLA','SLA Breach', 'Severe SLA Breach']

#Add conditions and values together
TasksWithSLA['SLA Status'] = np.select(conditions, values)

#Reduced SLA status dataframe
SLAStatus = TasksWithSLA[['Case ID','Department','SLA Status']]

#SLA status report
SLAStatusReport = pd.pivot_table(SLAStatus, values='Case ID', 
                                 index=['Department'],
                                 columns=['SLA Status'], 
                                 aggfunc='count',
                                fill_value=0,
                                margins=True).reset_index().rename(columns={'All':'Total cases per owner'})

#Add correct naming for totals
SLAStatusReport['Department']=SLAStatusReport['Department'].replace(['All'],'Total cases per SLA Status')

#Final report
SLAStatusReport.to_csv('SLA Report.csv')

SLAStatusReport

SLA Status,Department,SLA Breach,Severe SLA Breach,Within SLA,Total cases per owner
0,Christine,0,1,0,1
1,FDV,0,2,0,2
2,HR,2,9,1,12
3,Legal,0,1,0,1
4,Louise,0,4,0,4
5,RL,5,19,2,26
6,Total cases per SLA Status,7,36,3,46


#### Cases by type and region

In [22]:
#Add correct naming for department names
AllItemData['Main case']=AllItemData['Main case'].replace(['Performance'],['Poor Performance'])

#Summarize with pivot table cases by type and region
TypeAndRegionReport = pd.pivot_table(AllItemData, values='Case ID', 
                                     index=['Main case'],
                                     columns=['Region'], 
                                     aggfunc='count',
                                     margins=True).fillna(0).reset_index().rename(columns={'All':'Tota cases per case type',
                                                                                                           'Main case':'Case Type'})
#Add correct naming for totals column
TypeAndRegionReport['Case Type']=TypeAndRegionReport['Case Type'].replace(['All'],'Total case per Region')

#Final report
TypeAndRegionReport.to_csv('Cases by type and region.csv')

TypeAndRegionReport

Region,Case Type,East,South,South West,West,Tota cases per case type
0,Absenteeism,0.0,1.0,0.0,0.0,1
1,Fraud,16.0,16.0,6.0,14.0,52
2,Poor Performance,2.0,0.0,0.0,1.0,3
3,Resignation,9.0,3.0,1.0,3.0,16
4,Total case per Region,27.0,20.0,7.0,18.0,72
